In [1]:
#Conventional One

In [71]:
#--------------------------------#
#         House-keeping          #
#--------------------------------#

import numpy
import math
import time
from scipy.stats import norm
from joblib import Parallel, delayed
import multiprocessing
import sys

In [72]:
#Number of Workers
num_cores = 1

In [73]:
# Grid for x
nx            = 1500;
xmin          = 0.1;
xmax          = 4.0;

In [74]:
# Grid for e: parameters for Tauchen
ne            = 15;
ssigma_eps    = 0.02058;
llambda_eps   = 0.99;
m             = 1.5;

In [75]:
# Utility function
ssigma        = 2;
bbeta         = 0.97;
T             = 10;

In [76]:
# Prices
r             = 0.07;
w             = 5;

In [77]:
# Initialize the grid for X
xgrid = numpy.zeros(nx)

In [78]:
# Initialize the grid for E and the transition probability matrix
egrid = numpy.zeros(ne)
P     = numpy.zeros((ne, ne))

In [79]:
# Initialize value function V
V     = numpy.zeros((T, nx, ne))

In [80]:
len(V) # Number of period

10

In [81]:
len(V[0]) # For each age, possible state values

1500

In [82]:
len(V[0][0]) # For each state x, possible shocks

15

In [83]:
#--------------------------------#
#         Grid creation          #
#--------------------------------#

# Function to construct the grid for capital (x)
size = nx;
xstep = (xmax - xmin) /(size - 1);
it = 0;
for i in range(0,nx):
    xgrid[i] = xmin + it*xstep;
    it = it+1;

In [84]:
xgrid # Possible state variable values

array([0.1       , 0.10260173, 0.10520347, ..., 3.99479653, 3.99739827,
       4.        ])

In [85]:
# Function to construct the grid for productivity (e) using Tauchen (1986)
size = ne;
ssigma_y = math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2)));
estep = 2*ssigma_y*m / (size-1);
it = 0;
for i in range(0,ne):
    egrid[i] = (-m*math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2))) + it*estep);
    it = it+1;

In [86]:
egrid # possible 15 shocks

array([-0.21883163, -0.18756997, -0.15630831, -0.12504664, -0.09378498,
       -0.06252332, -0.03126166,  0.        ,  0.03126166,  0.06252332,
        0.09378498,  0.12504664,  0.15630831,  0.18756997,  0.21883163])

In [87]:
# Function to construct the transition probability matrix for productivity (P) using Tauchen (1986)
mm = egrid[1] - egrid[0];
for j in range(0,ne):
    for k in range(0,ne):
        if (k == 0):
            P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps);
        elif (k == ne-1):
            P[j, k] = 1 - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
        else:
            P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps) - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);

In [88]:
mm

0.03126166114086759

In [89]:
P[0]

array([7.43180973e-01, 2.41899285e-01, 1.48081627e-02, 1.11484939e-04,
       9.42709976e-08, 8.52351523e-12, 1.11022302e-16, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [90]:
# Exponential of the grid e
for i in range(0,ne):
	egrid[i] = math.exp(egrid[i]);

In [92]:
egrid

array([0.80345699, 0.82897112, 0.85529546, 0.88245574, 0.91047851,
       0.93939115, 0.96922193, 1.        , 1.03175544, 1.06451929,
       1.09832356, 1.13320131, 1.16918661, 1.20631465, 1.2446217 ])

In [93]:
#--------------------------------#
#     Structure and function     #
#--------------------------------#

# Value function
VV = math.pow(-10.0, 5); # just power function -10^5

In [94]:
# Data structure of state and exogenous variables
class modelState(object):
    def __init__(self,ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r):
        self.ind		= ind # combination of state vars
        self.ne			= ne # number of point on e-grid (shock)
        self.nx			= nx # number of point on x-grid (wealth)
        self.T			= T #last age (death)
        self.age		= age #age
        self.P			= P #transition prob matrix
        self.xgrid		= xgrid #state var - maybe saving?
        self.egrid		= egrid #possible shock grid
        self.ssigma		= ssigma #utility fct CRRA coefficient
        self.bbeta		= bbeta #discount factor
        self.w			= w #wage
        self.r			= r #interest rate 

In [95]:
modelState

__main__.modelState

In [96]:
V # will save the result here

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [97]:
# Function that returns value for a given state
# ind: a unique state that corresponds to a pair (ie,ix) - ie - labor productivity shock!
def value_func(states): #a fct that receives a 'state object'

	ind = states.ind 
	ne = states.ne
	nx = states.nx
	T = states.T
	age = states.age
	P = states.P
	xgrid = states.xgrid
	egrid = states.egrid
	ssigma = states.ssigma
	bbeta = states.bbeta
	w = states.w
	r = states.r

#maht.floor - largest integer less than or equal to ind/ne
	ix = int(math.floor(ind/ne)); #just dividing
	ie = int(math.floor(ind%ne)); #remainder after ind divided by ne .. 17%5 = 2

	VV = math.pow(-10.0, 3) #-10^3 - bottom of utility I guess..?
	for ixp in range(0,nx): # - for every possible next wealth state,
        #1. Form a expectation about the future
		expected = 0.0; # calculate the expectation
		if(age < T-1): #before you die, 
			for iep in range(0,ne): # - for every possible next realization of the shock 
				expected = expected + P[ie, iep]*V[age+1, ixp, iep] # form an expectation - for last period... haha no error
        #If you are not in the last period, expectation is not zero
        #2. Possible consumption functions
		cons  = (1 + r)*xgrid[ix] + egrid[ie]*w - xgrid[ixp]; #current state interest rate, wage, next period saving
        
        #3. Utility calculation
		utility = math.pow(cons, (1-ssigma))/(1-ssigma) + bbeta*expected;
        
        #Boundary.. / Borrowing limit (incomplete market)
		if(cons <= 0):
			utility = math.pow(-10.0,5); #any big super negative number will work
		
		if(utility >= VV): # 최대값을 걸러내는 장소 .. 어떤 state에서 다음 어떤 state로 갈지 결정하는거지 
            #다음 state를 1로 했을 때 만약에 다음 state를 0이라고 잡은 value function보다 높으면 고 아니면 업데이트 안한다.
            #사실 이 function은 value function만 계산하고 optimal policy는 안보인다?
			VV = utility;

		utility = 0.0;

	return[VV]; #so for this function, for given state, it results in the maximum value that household can achieve!

In [98]:
#--------------------------------#
#     Life-cycle computation     #
#--------------------------------#

print(" ")
print("Life cycle computation: ")
print(" ")


start = time.time()

for age in reversed(range(0,T)): #starting from "T"

	# This function computes `value_func` in parallel for all the states
	results = Parallel(n_jobs=num_cores)(delayed(value_func)(modelState(ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r)) for ind in range(0,nx*ne))

	# I write the results on the value matrix: V
	for ind in range(0,nx*ne): # for every case for shocks and state vars
		
		ix = int(math.floor(ind/ne)); # current state vars 
		ie = int(math.floor(ind%ne)); # current shocks

		V[age, ix, ie] = results[ind][0];

	finish = time.time() - start
	print("Age: ", age+1, ". Time: ", round(finish, 4), " seconds.")

finish = time.time() - start
print("TOTAL ELAPSED TIME: ", round(finish, 4), " seconds. \n")


#---------------------#
#     Some checks     #
#---------------------#

print(" - - - - - - - - - - - - - - - - - - - - - \n")
print("The first entries of the value function: \n")

for i in range(0,3):
	print(round(V[0, 0, i], 5)) # 소수점 5번째 자리까지만 표시

print(" \n")

 
Life cycle computation: 
 
Age:  10 . Time:  65.1433  seconds.
Age:  9 . Time:  463.7921  seconds.
Age:  8 . Time:  847.9525  seconds.
Age:  7 . Time:  1231.3666  seconds.


KeyboardInterrupt: 

In [101]:
result

NameError: name 'result' is not defined

In [100]:
nx*ne

22500

In [102]:
V_ow

NameError: name 'V_ow' is not defined

In [99]:
V

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [ ]:
# Faster Version with numba

In [ ]:
#--------------------------------#
#         House-keeping          #
#--------------------------------#

from numba import jit, jitclass, njit, prange, int64, float64
import numpy
import math
import time
from scipy.stats import norm
from collections import OrderedDict
import sys


In [32]:

#--------------------------------#
#         Initialization         #
#--------------------------------#

# Number of workers

# Grid for x
nx            = 1500;
xmin          = 0.1;
xmax          = 4.0;

# Grid for e: parameters for Tauchen
ne            = 15;
ssigma_eps    = 0.02058;
llambda_eps   = 0.99;
m             = 1.5;

# Utility function
ssigma        = 2;
bbeta         = 0.97;
T             = 10;

# Prices
r             = 0.07;
w             = 5;

# Initialize the grid for X
xgrid = numpy.zeros(nx)

# Initialize the grid for E and the transition probability matrix
egrid = numpy.zeros(ne)
P     = numpy.zeros((ne, ne))




#--------------------------------#
#         Grid creation          #
#--------------------------------#

# Function to construct the grid for capital (x)
size = nx;
xstep = (xmax - xmin) /(size - 1);
it = 0;
for i in range(0,nx):
	xgrid[i] = xmin + it*xstep;
	it = it+1;


# Function to construct the grid for productivity (e) using Tauchen (1986)
size = ne;
ssigma_y = math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2)));
estep = 2*ssigma_y*m / (size-1);
it = 0;
for i in range(0,ne):
	egrid[i] = (-m*math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2))) + it*estep);
	it = it+1;


# Function to construct the transition probability matrix for productivity (P) using Tauchen (1986)
mm = egrid[1] - egrid[0];
for j in range(0,ne):
	for k in range(0,ne):
		if (k == 0):
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps);
		elif (k == ne-1):
			P[j, k] = 1 - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
		else:
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps) - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);


# Exponential of the grid e
for i in range(0,ne):
	egrid[i] = math.exp(egrid[i]);



#--------------------------------#
#     Structure and function     #
#--------------------------------#

# Value function
VV = math.pow(-10.0, 5);

specs = OrderedDict()
specs['ind'] = int64
specs['ne'] = int64
specs['nx'] = int64
specs['T'] = int64
specs['age'] = int64
specs['P'] = float64[:,:]
specs['xgrid'] = float64[:]
specs['egrid'] = float64[:]
specs['ssigma'] = float64
specs['bbeta'] = float64
specs['w'] = float64
specs['r'] = float64
specs['V'] = float64[:,:,:]


# Data structure of state and exogenous variables
@jitclass(specs)
class modelState(object):
	def __init__(self,ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r,V):
		self.ind		= ind
		self.ne			= ne
		self.nx			= nx
		self.T			= T
		self.age		= age
		self.P			= P
		self.xgrid		= xgrid
		self.egrid		= egrid
		self.ssigma		= ssigma
		self.bbeta		= bbeta
		self.w			= w
		self.r			= r
		self.V			= V

# Function that returns value for a given state
# ind: a unique state that corresponds to a pair (ie,ix)
@njit
def value_func(states):

	ind = states.ind
	ne = states.ne
	nx = states.nx
	T = states.T
	age = states.age
	P = states.P
	xgrid = states.xgrid
	egrid = states.egrid
	ssigma = states.ssigma
	bbeta = states.bbeta
	w = states.w
	r = states.r
	V = states.V

	ix = int(math.floor(ind/ne));
	ie = int(math.floor(ind%ne));

	VV = math.pow(-10.0, 3)
	for ixp in range(0,nx):
		expected = 0.0;
		if(age < T-1):
			for iep in range(0,ne):
				expected = expected + P[ie, iep]*V[age+1, ixp, iep]

		cons  = (1 + r)*xgrid[ix] + egrid[ie]*w - xgrid[ixp];

		utility = math.pow(cons, (1-ssigma))/(1-ssigma) + bbeta*expected;

		if(cons <= 0):
			utility = math.pow(-10.0,5);
		
		if(utility >= VV):
			VV = utility;

		utility = 0.0;

	return[VV];



#--------------------------------#
#     Life-cycle computation     #
#--------------------------------#

print(" ")
print("Life cycle computation: ")
print(" ")

@njit(parallel=True)
def compute(age, V):

	for ind in prange(0,nx*ne):

		states = modelState(ind, ne, nx, T, age, P, xgrid, egrid, ssigma, bbeta, w, r, V)
		
		ix = int(math.floor(ind/ne));
		ie = int(math.floor(ind%ne));

		V[age, ix, ie] = value_func(states)[0];

	return(V)



start = time.time()
# Initialize value function V
V     = numpy.zeros((T, nx, ne))

for age in range(T-1, -1, -1):
	V = compute(age, V)

	finish = time.time() - start
	print("Age: ", age+1, ". Time: ", round(finish, 4), " seconds.")


finish = time.time() - start
print("TOTAL ELAPSED TIME: ", round(finish, 4), " seconds. \n")


#---------------------#
#     Some checks     #
#---------------------#

print(" - - - - - - - - - - - - - - - - - - - - - \n")
print("The first entries of the value function: \n")

for i in range(0,3):
	print(round(V[0, 0, i], 5))

print(" \n")

 
Life cycle computation: 
 
Age:  10 . Time:  1.63  seconds.
Age:  9 . Time:  2.007  seconds.
Age:  8 . Time:  2.379  seconds.
Age:  7 . Time:  2.8048  seconds.
Age:  6 . Time:  3.1679  seconds.
Age:  5 . Time:  3.5249  seconds.
Age:  4 . Time:  3.9079  seconds.
Age:  3 . Time:  4.2948  seconds.
Age:  2 . Time:  4.6808  seconds.
Age:  1 . Time:  5.0538  seconds.
TOTAL ELAPSED TIME:  5.0558  seconds. 

 - - - - - - - - - - - - - - - - - - - - - 

The first entries of the value function: 

-2.11762
-2.07729
-2.02366
 

